<a href="https://colab.research.google.com/github/ykamen/CS4342/blob/main/hw1part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!python --version
import numpy as np
import matplotlib.patches as patches
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from google.colab import drive

Python 3.7.13


In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
faces = np.load('drive/MyDrive/trainingFaces.npy', 'r')
labels = np.load('drive/MyDrive/trainingLabels.npy', 'r')

In [5]:
fPC(labels, jMap(faces,7)[:,9])

0.4705

In [6]:
print(faces[0][575])

-0.6677621653616024


One predictors comparison to all other pixels

In [176]:
def p13(x,k):
  return np.tile(x,(k,1))

def allMap(trset):
  map = jMap(trset,0)
  for i in range(1, trset.shape[1]):
    a = jMap(trset,i)
    map = np.dstack((map,a))
  return map

def jMap(trset,p1):
  vector = trset[:,p1]
  vmaptemp = p13(vector,trset.shape[1])
  vmap = vmaptemp.T
  return vmap > trset
#returns matrix showing the result of comparing pixel 1 to every other pixel
#across all available images. jmap

def fPC (y, yhat):
  return np.mean(y == yhat)

def bestJ(jmap,y):
  pc = 0
  pos = 0
  for i in range(jmap.shape[1]):
    temp = fPC(y, jmap[:,i])
    if(temp > pc):
      pc = temp
      pos = i
  return pc, pos
#returns the best predictor in one family of matrices e.g. best p1 > p2 
#for a single p1.

def bestFirstJ(trset,y):
  pc = 0
  p1 = 0
  p2 = 0
  for i in range(trset.shape[1]):
    temp_pc, temp_p2 = bestJ(jMap(trset,i),y)
    if(temp_pc > pc):
      pc = temp_pc
      p2 = temp_p2
      p1 = i
  return jMap(trset,p1)[:,p2]
#returns true/false values array

def jsAcc(y,js):
  c = 0
  for i in range(js.shape[0]):
    if (np.mean(js[i]) <= .5 and y[i]== False):
      c+=1
    elif (np.mean(js[i]) > .5 and y[i]== True):
      c+=1
  return c/y.shape[0]

def findNextJ(map,js,y):
  pc = 0
  f_p1 = 0
  f_p2 = 0
  for p1 in range(map.shape[1]):
    for p2 in range(map.shape[1]):
      a = np.column_stack((map[:,p2,p1],js))
      temp_pc = jsAcc(y,a)
      print(temp_pc)
      if (temp_pc>pc):
        pc = temp_pc
        f_p1 = p1
        f_p2 = p2
  return temp_pc,f_p1,f_p2

def stepwiseRegression(trF, trL, teF, teL):
  j = bestFirstJ(trF,trL)
  map = allMap(trF)
  for i in range(5):
    pc,p1,p2 = findNextJ(map,j,trL)
    j = np.column_stack((map[:,p2,p1],j))
  return j

#10 images took 24 minutes. and this is Friday 10:27PM. I am sorry but I won't
#be doing the n step 200. I will just leave the code here

def nAnalysis(trF,trL,teF,teL):
  print("n-images | training Accuracy | testing Accuracy")
  for n in range(400,2200,200):
    predictor = stepwiseRegression(trF[:n],trL[:n],teF[:n],teL[:n])
    print("%d | %d | %d" % (n, jsAcc(trL,predictor), jsAcc(teL,predictor)))
  return

IndexError: ignored